# Music Video Synthesis
* Extract lyrics from song with timestamps
* Compose scenes, include timestamps
* Generate images for each scene
* A human should evalute photos and scenes, creating a curated one with the desired characteristics
* Construct video text prompt for each scene
* Build videos for each scene, use referall link to sign up: https://www.segmind.com/invite/773118b7-41f4-4154-87f4-49326d973ec3
* Stitch together

# We will use openai whipser for stability

In [1]:
#!pip install --quiet --upgrade pip
#!pip3 install torch torchvision torchaudio optimum-quanto torchao --index-url https://download.pytorch.org/whl/cu126
#!pip install --quiet --upgrade openai-whisper openai
# Ubuntu or Debian
#!sudo apt update && sudo apt install ffmpeg
#!pip install setuptools-rust
#!pip install -U diffusers imageio imageio_ffmpeg opencv-python moviepy transformers huggingface-hub optimum pillow safetensors optimum-quanto accelerate
#!pip install git+https://github.com/xhinker/sd_embed.git@main
#!pip install accelerate flash_attention numba -U
#!pip install flash_attn --no-build-isolation
#!pip install -r requirements.txt -U

In [2]:
import argparse
import base64
import cv2
import diffusers
import gc
import imageio
import imageio_ffmpeg
import json
import math
import moviepy as mp
import numpy as np
import os
import psutil
import random
import requests
import sys
import tempfile
import time
import transformers
import torch
import torch.multiprocessing as mp
import whisper

from contextlib import contextmanager
from datetime import datetime, timedelta
from diffusers import AutoencoderKL, AutoPipelineForText2Image
from diffusers import FlowMatchEulerDiscreteScheduler
from diffusers import EulerDiscreteScheduler, EulerAncestralDiscreteScheduler, DPMSolverMultistepScheduler, PNDMScheduler, DDIMScheduler
from diffusers.image_processor import VaeImageProcessor
from diffusers.pipelines.flux.pipeline_flux import FluxPipeline
from diffusers.models.transformers.transformer_flux import FluxTransformer2DModel
from diffusers.utils import export_to_video, load_video, load_image
from hyvideo.utils.file_utils import save_videos_grid
from hyvideo.config import parse_args
from hyvideo.inference import HunyuanVideoSampler
from hyvideo.constants import NEGATIVE_PROMPT
from mmgp import offload, profile_type
from huggingface_hub import hf_hub_download, snapshot_download
from mmgp import offload, profile_type
from numba import cuda
from openai import OpenAI
from optimum.quanto import freeze, qfloat8, quantize, requantize
from pathlib import Path
from PIL import Image
from safetensors.torch import load_file as load_safetensors, save_file as save_safetensors
from sd_embed.embedding_funcs import get_weighted_text_embeddings_flux1
from torchao.quantization import quantize_, int8_weight_only, int8_dynamic_activation_int8_weight
from transformers import CLIPTextModel, CLIPTokenizer, T5TokenizerFast, T5EncoderModel
from transformers import CLIPImageProcessor, CLIPVisionModelWithProjection

os.environ["TOKENIZERS_PARALLELISM"] = "false"
# Define the paths where quantized weights will be saved

dtype = torch.bfloat16
MAX_SEED = np.iinfo(np.int32).max
device = "cuda" if torch.cuda.is_available() else "cpu"
retry_limit = 3
quantization = int8_weight_only

WIDTH = 848
HEIGHT = 480

2025-02-02 11:01:31.000494: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-02 11:01:31.153081: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738515691.213136     905 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738515691.230773     905 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-02 11:01:31.383592: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
# Configuration
CONFIG = {
    "openai_api_key": "",
    "openai_model": "gpt-4o-mini",
    "openai_model_large": "gpt-4o",
    "hf_token": "",
    "base_working_dir": "./images",
    "base_video_dir": "./output",
    "audio_files": [
        #"/mnt/d/Share/Audio/DidgeridooUnity.mp3",
        "/mnt/d/Share/Audio/WingsOfLight.mp3",
        "/mnt/d/Share/Audio/WingsOfLight.mp3",
        "/mnt/d/Share/Audio/WingsOfLight.mp3",
        # Add more audio file paths here
    ],
    "device": device,
    "dtype": dtype,
    "retry_limit": retry_limit,
    "MAX_SEED": MAX_SEED
}

# Ensure base directories exist
os.makedirs(CONFIG["base_working_dir"], exist_ok=True)
os.makedirs(CONFIG["base_video_dir"], exist_ok=True)

args = argparse.Namespace(
    quantize_transformer=False,
    lora_weight=[],
    lora_multiplier=[],
    profile=-1,
    verbose=1,
    server_port=0,
    server_name='',
    open_browser=False,
    model='HYVideo-T/2-cfgdistill',
    latent_channels=16,
    precision='bf16',
    rope_theta=256,
    vae='884-16c-hy',
    vae_precision='fp16',
    vae_tiling=True,
    text_encoder='llm',
    text_encoder_precision='fp16',
    text_states_dim=4096,
    text_len=256,
    tokenizer='llm',
    prompt_template='dit-llm-encode',
    prompt_template_video='dit-llm-encode-video',
    hidden_state_skip_layer=2,
    apply_final_norm=False,
    text_encoder_2='clipL',
    text_encoder_precision_2='fp16',
    text_states_dim_2=768,
    tokenizer_2='clipL',
    text_len_2=77,
    denoise_type='flow',
    flow_shift=7.0,
    flow_reverse=True,
    flow_solver='euler',
    use_linear_quadratic_schedule=False,
    linear_schedule_end=25,
    model_base='ckpts',
    dit_weight='ckpts/hunyuan-video-t2v-720p/transformers/mp_rank_00_model_states.pt',
    model_resolution='540p',
    load_key='module',
    use_cpu_offload=False,
    batch_size=1,
    infer_steps=50,
    disable_autocast=False,
    save_path='./results',
    save_path_suffix='',
    name_suffix='',
    num_videos=1,
    video_size=(720, 1280),
    video_length=129,
    prompt=None,
    seed_type='auto',
    seed=None,
    neg_prompt=None,
    cfg_scale=1.0,
    embedded_cfg_scale=6.0,
    reproduce=False,
    ulysses_degree=1,
    ring_degree=1
)

In [4]:
class SamplerArgs:
    """
    Minimal container for sampler-related settings.
    Extend this if you need additional fields that HunyuanVideoSampler
    or your pipeline expects.
    """
    def __init__(self):
        # Set to True if you want flow reversal in the pipeline
        self.flow_reverse = True    
def reset_memory(device):
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats(device)
    torch.cuda.reset_accumulated_memory_stats(device)
    torch.cuda.synchronize()
    
def get_openai_prompt_response(
    prompt: str,
    config: dict,
    max_tokens: int = 6000,
    temperature: float = 0.33,
    openai_model: str = "",
):
    """
    Sends a prompt to OpenAI's API and retrieves the response with retry logic.
    """
    client = OpenAI(api_key=config["openai_api_key"])
    response = client.chat.completions.create(
        max_tokens=max_tokens,
        messages=[
            {
                "role": "system",
                "content": """Act as a helpful assistant, you are an expert editor.""",
            },
            {"role": "user", "content": prompt},
        ],
        model=openai_model or config["openai_model"],
        temperature=temperature,
    )

    retry_count = 0
    while retry_count < config["retry_limit"]:
        try:
            message_content = response.choices[0].message.content
            return message_content
        except Exception as e:
            print(f"Error occurred: {e}")
            retry_count += 1
            if retry_count == config["retry_limit"]:
                print("Retry limit reached. Moving to the next iteration.")
                return ""
            else:
                print(f"Retrying... (Attempt {retry_count}/{config['retry_limit']})")
                time.sleep(1)  # Optional: wait before retrying


def load_flux_pipe():
    bfl_repo = "black-forest-labs/FLUX.1-dev"
    revision = "refs/pr/3"
    adapter_id = "alimama-creative/FLUX.1-Turbo-Alpha"

    scheduler = FlowMatchEulerDiscreteScheduler.from_pretrained(bfl_repo, subfolder="scheduler", revision=revision)
    text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=dtype)
    tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=dtype)
    text_encoder_2 = T5EncoderModel.from_pretrained(bfl_repo, subfolder="text_encoder_2", torch_dtype=dtype, revision=revision)
    tokenizer_2 = T5TokenizerFast.from_pretrained(bfl_repo, subfolder="tokenizer_2", torch_dtype=dtype, revision=revision)
    vae = AutoencoderKL.from_pretrained(bfl_repo, subfolder="vae", torch_dtype=dtype, revision=revision)
    transformer = FluxTransformer2DModel.from_pretrained(bfl_repo, subfolder="transformer", torch_dtype=dtype, revision=revision)
    
    quantize_(transformer, quantization())
    quantize_(text_encoder_2, quantization())
    pipe = FluxPipeline(
        scheduler=scheduler,
        text_encoder=text_encoder,
        tokenizer=tokenizer,
        text_encoder_2=text_encoder_2,
        tokenizer_2=tokenizer_2,
        vae=vae,
        transformer=transformer,
    )

    pipe = pipe.to('cuda')
    pipe.load_lora_weights(adapter_id)

    return pipe


def gen_flux_image(pipe, prompt, config: dict, height=1024, width=1024, guidance_scale=3.5, num_inference_steps=8, max_sequence_length=512, seed=-1):
    """
    Generates an image based on the provided prompt using the Flux pipeline.
    """
    if seed == -1:
        seed = random.randint(0, MAX_SEED)
        
    with torch.no_grad():
        prompt_embeds, pooled_prompt_embeds = get_weighted_text_embeddings_flux1(
            pipe        = pipe,
            prompt    = prompt
        )
        
        image = pipe(
            prompt_embeds               = prompt_embeds,
            pooled_prompt_embeds      = pooled_prompt_embeds,
            height=height,
            width=width,
            guidance_scale=guidance_scale,
            output_type="pil",
            num_inference_steps=num_inference_steps,
            max_sequence_length=max_sequence_length,
            generator=torch.Generator("cpu").manual_seed(seed)
        ).images[0]

        # Delete variables
        del prompt_embeds
        del pooled_prompt_embeds
        torch.cuda.empty_cache()

        return image


def image_file_to_base64(image_path):
    with open(image_path, 'rb') as f:
        image_data = f.read()
    return base64.b64encode(image_data).decode('utf-8')

# Use this function to fetch an image from a URL and convert it to base64
def image_url_to_base64(image_url):
    response = requests.get(image_url)
    image_data = response.content
    return base64.b64encode(image_data).decode('utf-8')
    
def load_hunyuan_video_sampler(
    server_config_filename: str = "gradio_config.json",
    forced_profile_no: int = -1,
    verbose_level: int = 1,
    quantize_transformer: bool = True,
    lora_weight: list = None,
    lora_multiplier: list = None,
    device: str = "cpu",
) -> HunyuanVideoSampler:
    """
    Loads the HunyuanVideo pipeline according to settings in `server_config_filename`.
    If `forced_profile_no` is >= 0, that overrides the 'profile' field in the server config.
    This version does NOT parse any command-line arguments.
    """
    if lora_weight is None:
        lora_weight = []
    if lora_multiplier is None:
        lora_multiplier = []

    # -----------------------------------------------------------------------
    # Read or create server_config
    # -----------------------------------------------------------------------
    if not Path(server_config_filename).is_file():
        # Default config if none present
        server_config = {
            "attention_mode": "sage",
            "transformer_filename": "ckpts/hunyuan-video-t2v-720p/transformers/hunyuan_video_720_quanto_int8.safetensors",
            "text_encoder_filename": "ckpts/text_encoder/llava-llama-3-8b-v1_1_quanto_int8.safetensors",
            "compile": "",
            "profile": profile_type.HighRAM_LowVRAM,
        }
        with open(server_config_filename, "w", encoding="utf-8") as writer:
            writer.write(json.dumps(server_config))
    else:
        with open(server_config_filename, "r", encoding="utf-8") as reader:
            text = reader.read()
        server_config = json.loads(text)

    # Pull out config
    transformer_filename = server_config["transformer_filename"]
    text_encoder_filename = server_config["text_encoder_filename"]
    attention_mode = server_config["attention_mode"]
    profile = forced_profile_no if forced_profile_no >= 0 else server_config["profile"]
    compile_mode = server_config.get("compile", "")

    # -----------------------------------------------------------------------
    # Download any missing models from HF or any other source (if needed)
    # -----------------------------------------------------------------------
    def download_models(transformer_filename, text_encoder_filename):
        """
        Stub: Implement your huggingface_hub logic here if needed.
        """
        pass

    download_models(transformer_filename, text_encoder_filename)

    # -----------------------------------------------------------------------
    # Optional: tweak VAE config, etc.
    # -----------------------------------------------------------------------
    offload.default_verboseLevel = verbose_level

    vae_config_path = "./ckpts/hunyuan-video-t2v-720p/vae/config.json"
    if os.path.isfile(vae_config_path):
        with open(vae_config_path, "r", encoding="utf-8") as reader:
            vae_config = json.loads(reader.read())
        # Example: reduce time window used by the VAE for temporal splitting
        if vae_config.get("sample_tsize", 64) == 64:
            vae_config["sample_tsize"] = 32
        with open(vae_config_path, "w", encoding="utf-8") as writer:
            writer.write(json.dumps(vae_config))

    # -----------------------------------------------------------------------
    # Decide how to pin memory, partial pin, etc. 
    # -----------------------------------------------------------------------
    if profile == 5:
        pinToMemory = False
        partialPinning = False
    else:
        pinToMemory = True
        physical_memory = psutil.virtual_memory().total
        # E.g. partial pin if <= 32 GB of RAM
        partialPinning = physical_memory <= (2**30) * 32

    # -----------------------------------------------------------------------
    # Load the pipeline
    # -----------------------------------------------------------------------
    hunyuan_video_sampler = HunyuanVideoSampler.from_pretrained(
        transformer_filename,
        text_encoder_filename,
        attention_mode=attention_mode,
        pinToMemory=pinToMemory,
        partialPinning=partialPinning,
        args=args,      # passes our simple SamplerArgs object
        device=device,
    )

    pipe = hunyuan_video_sampler.pipeline

    # -----------------------------------------------------------------------
    # Optionally load LoRAs
    # -----------------------------------------------------------------------
    if len(lora_weight) > 0:
        offload.load_loras_into_model(pipe.transformer, lora_weight, lora_multiplier)

    # -----------------------------------------------------------------------
    # Profile, compile, or quantize
    # -----------------------------------------------------------------------
    offload.profile(
        pipe,
        profile_no=profile,
        compile=compile_mode,
        quantizeTransformer=quantize_transformer,
    )

    return hunyuan_video_sampler


def generate_video(
    hunyuan_video_sampler,
    height=HEIGHT,
    width=WIDTH,
    video_length=121,
    infer_steps=50,
    prompt="A cat walks on the grass, realistic style.",
    negative_prompt="Aerial view, overexposed, low quality, deformation",
    flow_shift=7.0,
    filename="./output.mp4",
    seed=42,
    cfg_scale=7.5,
    batch_size=1,
    embedded_cfg_scale=1.0,
):

       # TeaCache
    trans = hunyuan_video_sampler.pipeline.transformer.__class__
    trans.enable_teacache = False
    if trans.enable_teacache:
        trans.num_steps = num_inference_steps
        trans.cnt = 0
        trans.rel_l1_thresh = 0.15 # 0.1 for 1.6x speedup, 0.15 for 2.1x speedup
        trans.accumulated_rel_l1_distance = 0
        trans.previous_modulated_input = None
        trans.previous_residual = None
        
    """
    Generates and saves a video using the provided sampler, based on the specified parameters.
    The result is written to 'filename'.
    """
    outputs = hunyuan_video_sampler.predict(
        prompt=prompt,
        height=480,
        width=848,
        video_length=121,
        seed=seed,
        negative_prompt=negative_prompt,
        infer_steps=50,
        guidance_scale=1.0,
        num_videos_per_prompt=1,
        flow_shift=7.0,
        batch_size=batch_size,
        embedded_guidance_scale=6.0,
    )

    samples = outputs["samples"]
    # Assuming one video per prompt:
    for i, sample in enumerate(samples):
        # shape is (C, T, H, W)
        sample = sample.unsqueeze(0)  # (1, C, T, H, W)
        save_videos_grid(sample, filename, fps=24)

    return filename


def unload_hunyuan_video_sampler(hunyuan_video_sampler):
    """
    Frees the memory used by the pipeline.
    In a normal Python script, deleting references and calling torch.cuda.empty_cache()
    is usually enough.
    """
    del hunyuan_video_sampler
    torch.cuda.empty_cache()


def create_scenes(text: str, video_summary: str, config: dict):
    """
    Creates scenes based on the extracted lyrics using OpenAI's API.
    """
    # Generate scenes JSON
    prompt = f'''Create a json list of diverse, unique scenes (groupings of text), scene_description (200 words or less), and action_sequence (30 words or less) from the following text.  Scenes should be groups of lyrics with new scenes when the lyric context changes.  Text: {text}   
The json list should have the start value for the first item in the scene and the text that is combined for all items in the same scene.  
The scene_description should include sensory rich details such as attire, setting, mood, lighting, and changing compositions, painting a clear visual scene consistent with the video theme and different from other scenes.  Use theme descriptions, such as graphic novel, water color, render, oil painting, etc.  Scenes should avoid depictions of literal people, unless they are close up of a single person.  Favor symbolism and artistic illustrations of concepts and feeling, emotion.  Avoid depections of literal people. Evoke strong emotions with atmospheric lighting and tone. Use descriptive phrases to capture the mood (e.g., flickering neon, golden twilight, ethereal glow).
Avoid scenes with many people moving.
The action_sequence should describe the action in the scene.  Scenes should be unique, creative, imaginative, and awe-inspiring to create an amazing video.  Create beautiful and mesmerizing scene descriptions that are creative, unique, artistic, and imaginative. Each scene must be unique, imaginative, and visually captivating, blending creativity with artistic flair. Use powerful, descriptive language to craft scenes that are awe-inspiring and leave the audience in wonder. These scenes should evoke a sense of beauty, grandeur, mystery, or anything emotional, drawing from both realistic and fantastical elements. Ensure the descriptions are immersive, emotionally resonant, and filled with unexpected twists that engage the senses and imagination, suitable for creating a stunning, cinematic video experience.  Use descriptions of special effects in the scenes.  
Action should avoid sudden or fast movement or zooms, avoid any fast camera movement.  Avoid human movements like walking, dancing, shopping, etc.
Return only the json list, less jargon. The json list fields should be: start, text, scene_description, action_sequence'''

    result = get_openai_prompt_response(prompt, config, openai_model=config["openai_model"], temperature=0.66)
    result = result.replace("```", "").replace("```json\n", "").replace("json\n", "").replace("\n", "")
    scenes = json.loads(result)
    return scenes

def revise_scenes(scenes, config: dict):
    """
    Revise scenes based on the extracted scenes.
    """
    # Generate scenes JSON
    prompt = f'''Revise the JSON scenes to update the scene_description and action_sequence to engage the senses and imagination, suitable for creating a stunning, cinematic video experience.  We want unique scenes, even ones in the same sequence. Use descriptions of special effects in the scenes.  JSON scenes: {scenes}   
The scene_description (200 words or less) should include details such as attire, setting, mood, lighting, and any significant movements or expressions, painting a clear visual scene consistent with the video theme and different from other scenes. Use theme descriptions, such as graphic novel, water color, render, oil painting, etc.  Scenes should avoid depictions of literal people, unless they are close up of a single person.  Favor symbolism and artistic illustrations of concepts and feeling, emotion.  Avoid depections of literal people. Evoke strong emotions with atmospheric lighting and tone. Use descriptive phrases to capture the mood (e.g., flickering neon, golden twilight, ethereal glow).
The action_sequence (30 words or less) should describe the action in the scene.  The goal is to create input to create a stunning, cinematic video experience.   
Action should avoid sudden or fast movement or zooms, avoid any fast camera movement. Avoid human movements like walking, dancing, shopping, etc.
Only update the scene_description and action_sequence. We do not want to have similar scene_descriptions and action_sequences for consecutive scenes, we want unique scenes that tell a brilliant, cohesive story.  Please update the scene_description and action_sequence to be differemt, creative, and consistent.  
Do not delete any items as having scenes with the given start times are important. 
Return only the json list, less jargon. The json list fields should be: start, text, scene_description, action_sequence'''

    result = get_openai_prompt_response(prompt, config, openai_model=config["openai_model"], temperature=0.33)
    result = result.replace("```", "").replace("```json\n", "").replace("json\n", "").replace("\n", "")
    scenes = json.loads(result)
    return scenes


def process_audio_scenes(audio_file: str, config: dict):
    # set maximum duration for an image basis, should be in intervals of video generation length
    video_gen_length = 5
    max_duration_seconds  = video_gen_length * 3
    """
    Processes a single audio file through the entire workflow.
    """
    # Create unique identifier based on audio file name
    audio_basename = os.path.splitext(os.path.basename(audio_file))[0]
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    unique_id = f"{audio_basename}_{timestamp}"

    # Create unique directories for images and videos
    print(f"Create unique directories for images and videos")
    audio_images_dir = os.path.join(config["base_working_dir"], unique_id)
    audio_videos_dir = os.path.join(config["base_video_dir"], unique_id)
    os.makedirs(audio_images_dir, exist_ok=True)
    os.makedirs(audio_videos_dir, exist_ok=True)

    # Step 1: Transcribe audio using Whisper
    print(f"Transcribe audio using Whisper")
    model = whisper.load_model("turbo")
    result = model.transcribe(audio_file)

    # Cleanup Whisper model memory
    del model
    gc.collect()
    torch.cuda.empty_cache()

    segments = result['segments']

    # Extract list of start times and texts
    segment_texts_and_start_times = [(segment['text'].strip(), segment['start']) for segment in segments]

    # Combine texts
    text = ""
    for segment_text, start in segment_texts_and_start_times:
        text += f"Start: {start}, Text: {segment_text}\n"

    last_end_value = segments[-1]['end']

    # Path to scenes.json file
    scenes_file_path = os.path.join(audio_images_dir, "scenes.json")

    # Check if scenes.json exists
    if os.path.exists(scenes_file_path):
        print(f"Scenes file already exists at {scenes_file_path}. Skipping scene generation.")
        with open(scenes_file_path, "r") as scenes_file:
            scenes = json.load(scenes_file)
        return scenes, audio_images_dir, audio_videos_dir, last_end_value

    # Step 2: Generate video summary using OpenAI
    print(f"Generate video summary using OpenAI")
    video_summary_prompt = f'Create a short summary that describes a music video based on these lyrics: {text}'
    video_summary = get_openai_prompt_response(video_summary_prompt, config, openai_model=config["openai_model"])

    # Step 3: Create scenes based on lyrics
    print(f"Create scenes based on lyrics")
    try:
        scenes = create_scenes(text, video_summary, config)
    except:
        try:
            scenes = create_scenes(text, video_summary, config)
        except:
            try:
                scenes = create_scenes(text, video_summary, config)
            except: 
                return "", audio_images_dir, audio_videos_dir, last_end_value
            
    # we don't want scenes longer than 18 seconds
    new_scenes = []
    for i in range(len(scenes)):
        scene = scenes[i]
        if i == 0:
            start_time = 0
        else:
            start_time = scene['start']
        # Determine the end time
        if i < len(scenes) - 1:
            end_time = scenes[i + 1]['start']
        else:
            end_time = last_end_value
        duration = end_time - start_time
        # Split the scene if duration exceeds max_duration_seconds seconds
        while duration > max_duration_seconds:
            new_scene = scene.copy()
            new_scene['start'] = start_time
            new_scenes.append(new_scene)
            start_time += max_duration_seconds
            duration = end_time - start_time
        # Append the remaining part of the scene
        if duration > 0:
            new_scene = scene.copy()
            new_scene['start'] = start_time
            new_scenes.append(new_scene)
    # Replace the original scenes with the new list
    scenes = new_scenes
    # improve the scenes with a revision
    try:
        scenes_revised = revise_scenes(scenes, config)
        scenes = scenes_revised
        print(f'revised scenes')
    except:
        try:
            scenes_revised = revise_scenes(scenes, config)
            scenes = scenes_revised
            print(f'revised scenes')
        except:
            print('cannot revise scenes')
            
    
    # Save the scenes to scenes.json
    with open(scenes_file_path, "w") as scenes_file:
        json.dump(scenes, scenes_file)
        
    return scenes, audio_images_dir, audio_videos_dir, last_end_value, timestamp

def process_audio_images(config: dict, scenes, audio_images_dir):
    # Step 4: Load Flux pipeline and generate images
    print(f"Load Flux pipeline and generate images")
    flux_pipe = load_flux_pipe()
    height = HEIGHT
    width = WIDTH
    guidance_scale = 3.9
    num_inference_steps = 8
    max_sequence_length = 512
    seed = -1

    try:
        # Generate images for each scene
        image_num = 1
        for scene in scenes:
            image_prompt = scene['scene_description']
            image = gen_flux_image(flux_pipe, image_prompt, config, height, width, guidance_scale, num_inference_steps, max_sequence_length, seed)
            filename = f"image_{str(image_num).zfill(2)}.jpg"
            image_path = os.path.join(audio_images_dir, filename)
            image.save(image_path, dpi=(300, 300))
            del image
            torch.cuda.empty_cache()
            image_num += 1
    finally:
        # Move the pipeline back to CPU and delete it
        flux_pipe.to('cpu')
        del flux_pipe
        gc.collect()
        torch.cuda.empty_cache()
    return

def process_audio_video(config: dict, scenes, audio_images_dir, audio_videos_dir, last_end_value, timestamp, skip_first):
    video_num = 1
    negative_prompt = "Aerial view, aerial view, overexposed, low quality, deformation, a poor composition, bad hands, bad teeth, bad eyes, bad limbs, distortion"
   
    hunyuan_video_sampler = load_hunyuan_video_sampler(
        server_config_filename="gradio_config.json",
        forced_profile_no=-1,
        verbose_level=1,
        quantize_transformer=True,
        lora_weight=[],
        lora_multiplier=[],
        device="cuda",  # or "cpu"
    )
    # Step 7: Generate video sequences
    for i, scene in enumerate(scenes):
        prompt = scene["scene_description"] + " " + scene["action_sequence"]

        # Calculate duration to keep the video in 6-second increments
        if i + 1 < len(scenes):
            next_start_time = scenes[i + 1]["start"]
        else:
            next_start_time = last_end_value  # Use the final ending time for the last scene

        if i == 0:
            duration = next_start_time
        else:
            duration = next_start_time - scene["start"]
        num_video_segments = int((duration + 2) // 5)

        print(f"Scene {i+1} has {num_video_segments} segments")
        for j in range(num_video_segments):
            video_name = f"video_{str(video_num).zfill(2)}_{str(i+1).zfill(2)}_{str(j+1).zfill(2)}_{timestamp}.mp4"
            video_output_path = os.path.join(audio_videos_dir, video_name)
            if video_num > skip_first:
                seed = random.randint(0, MAX_SEED)
                generate_video(hunyuan_video_sampler=hunyuan_video_sampler, height=HEIGHT, width=WIDTH, video_length=121, infer_steps=50,
                    prompt=prompt, negative_prompt=negative_prompt, flow_shift=7.0, filename=video_output_path,
                    seed=seed, cfg_scale=7.5, batch_size=1, embedded_cfg_scale=1.0)
                
                time.sleep(1)  # Pause for 1 second

            video_num += 1  # Increment video number for the next segment
    
    unload_hunyuan_video_sampler(hunyuan_video_sampler)
    return


def process_all_audios(audio_file, config: dict):
    """
    Processes a list of audio files through the workflow.
    """
    print(f"Processing audio file: {audio_file}")
    scenes, audio_images_dir, audio_videos_dir, last_end_value, timestamp = process_audio_scenes(audio_file, config)
    print(f'{len(scenes)} scenes:\n{json.dumps(scenes, indent=4)}')
    # Create starting images for scenes
    process_audio_images(config, scenes, audio_images_dir)
    return config, scenes, audio_images_dir, audio_videos_dir, last_end_value, timestamp

def create_video(images_only):
    config, scenes, audio_images_dir, audio_videos_dir, last_end_value, timestamp = process_all_audios(audio_file, CONFIG)
    if not images_only:
        process_audio_video(config, scenes, audio_images_dir, audio_videos_dir, last_end_value, timestamp, 0)
    print(f'audio_images_dir: {audio_images_dir}')
    print(f'audio_videos_dir: {audio_videos_dir}')
    print(f'last_end_value: {last_end_value}')
    print(f'timestamp: {timestamp}')
    
    return
    


In [5]:
# run and curate images for scenes
human_in_loop = False
for audio_file in CONFIG["audio_files"]:
    create_video(human_in_loop)

reset_memory(device)

Processing audio file: /mnt/d/Share/Audio/WingsOfLight.mp3
Create unique directories for images and videos
Transcribe audio using Whisper


/home/silwa/anaconda3/lib/python3.11/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Generate video summary using OpenAI
Create scenes based on lyrics
revised scenes
25 scenes:
[
    {
        "start": 0,
        "text": "Angels. Some see them as baby-faced protectors. Others envision them locked in battle with demons. But what if they're more than our eyes can handle?",
        "scene_description": "A vast celestial expanse unfolds, where soft, golden twilight mingles with swirling iridescent clouds. Ethereal beings, adorned in flowing robes of shimmering light, hover gracefully, their wings glistening like dew-kissed petals. The air is thick with a palpable serenity, as gentle whispers of hope echo through the atmosphere. Shadows flicker playfully, hinting at unseen conflicts, yet a profound tranquility reigns. The palette is a harmonious blend of pastel blues and warm yellows, creating a watercolor dreamscape that invites exploration of divine mysteries.",
        "action_sequence": "Angels glide softly through the air, their luminous forms illuminating hidden pathw

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


...

audio_images_dir: ./images/WingsOfLight_20250202_110141
audio_videos_dir: ./output/WingsOfLight_20250202_110141
last_end_value: 365.04
timestamp: 20250202_110141
Processing audio file: /mnt/d/Share/Audio/WingsOfLight.mp3
Create unique directories for images and videos
Transcribe audio using Whisper


/home/silwa/anaconda3/lib/python3.11/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Generate video summary using OpenAI
Create scenes based on lyrics
revised scenes
26 scenes:
[
    {
        "start": 0,
        "text": "Angels, some see them as baby-faced protectors, others envision them locked in battle with demons.",
        "scene_description": "In a dreamlike watercolor world, soft pastels swirl into a twilight sky where luminous angels glide through ethereal clouds. Their wings shimmer with a celestial glow, casting gentle light on the encroaching shadows of demons below. The atmosphere is thick with tension, as beams of hope pierce the darkness, illuminating the serene yet haunting landscape, where chaos and tranquility coexist.",
        "action_sequence": "Angels unfurl their wings, radiating soft light that encircles the lurking shadows, creating a dance of illumination and darkness."
    },
    {
        "start": 15,
        "text": "Angels, some see them as baby-faced protectors, others envision them locked in battle with demons.",
        "scene_descripti

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

HfHubHTTPError: 503 Server Error: Service Temporarily Unavailable for url: https://huggingface.co/api/models/black-forest-labs/FLUX.1-dev/revision/refs%2Fpr%2F3

## Video is expensive, only process after curating scenes and images

In [ ]:
human_in_loop = True
skip_first = 0
if human_in_loop:
    scenes_file_path = './images/RememberUs_20250122_202916/scenes.json'
    audio_images_dir = './images/RememberUs_20250122_202916'
    audio_videos_dir = './output/RememberUs_20250122_202916'
    timestamp = '20250122_202916'
    last_end_value = 156.0
    
    with open(scenes_file_path, "r") as scenes_file:
        scenes = json.load(scenes_file)
    process_audio_video(CONFIG, scenes, audio_images_dir, audio_videos_dir, last_end_value, timestamp, skip_first)